In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(588, with_units=False)
Hist_table


Month,Entries
IntegerAndSexagesimal,Temporal
31 ;,"29 ; 12,44,03"
59 ;,"59 ; 01,28,06"
90 ;,"88 ; 14,12,09"
120 ;,"118 ; 02,56,12"
151 ;,"147 ; 15,40,15"
181 ;,"177 ; 04,24,18"
212 ;,"206 ; 17,08,21"
243 ;,"236 ; 05,52,24"
273 ;,"265 ; 18,36,27"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Temporal('0;0,0,0')
    Delta_recomp[i][1]=Temporal('0;0,0,0')

In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

#length of the synodic month
synodic_month=REVO/daily_mean
synodic_month=Temporal(synodic_month,7)

for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=5):
        time=((i+1)*synodic_month)%365
                    
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*time
            Delta_recomp[i][1]=24*60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Month,Entries
IntegerAndSexagesimal,Temporal
31 ;,"29 ; 12,44,03"
59 ;,"59 ; 01,28,06"
90 ;,"88 ; 14,12,09"
120 ;,"118 ; 02,56,12"
151 ;,"147 ; 15,40,15"
181 ;,"177 ; 04,24,18"
212 ;,"206 ; 17,08,21"
243 ;,"236 ; 05,52,24"
273 ;,"265 ; 18,36,27"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.0, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
Month,
31 ;,0
59 ;,0
90 ;,0
120 ;,0
151 ;,0
181 ;,0
212 ;,0
243 ;,0
273 ;,0


In [13]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygies dates months.csv')
B.to_csv('Mean syzygies dates months.zip', index=False, compression=compression_opts)